In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env

m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "damodar",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "damodar", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "powder_river_basin_cbm"}

m = wt.case_study_trains.get_case_study(m=m)


damodar

------- Adding Unit Processes -------
onsite_storage
microfiltration
reverse_osmosis
uv_aop
chlorination
municipal_drinking
waiv
-------------------------------------

adding source

Connecting unit processes...


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

In [4]:
### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')
        
wt.run_water_tap(m=m, objective=True)

# prints RO results
print("\nOptimal RO area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print(f'\nSetting feed presure for {key} to {m.fs.pfd_dict[key]["Parameter"]["feed_pressure"]} bar (user input)...\n')

wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

Unfixing feed presure and area for reverse_osmosis ...

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------

Optimal RO area and pressures:
	Feed pressure for reverse_osmosis: 28.89 bar
	Membrane area for reverse_osmosis: 7543.15 m2

Setting feed presure for reverse_osmosis to 85 bar (user input)...

----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


ONSITE STORAGE:

	total cap investment: 2.86121
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.03782

MICROFILTRATION:

	total cap investment: 5.09541
	cat and chem cost: 0.0
	electricity cost: 0.03405
	total fixed op cost: 0.06785

REVERSE OSMOSIS:

	total cap investment: 1.81092
	cat and chem c

In [5]:
# If you need the system recovery to match better.... set a maximum recovery rate.

In [6]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [7]:
wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


ONSITE STORAGE:

	total cap investment: 2.86121
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.03782

MICROFILTRATION:

	total cap investment: 5.09541
	cat and chem cost: 0.0
	electricity cost: 0.03405
	total fixed op cost: 0.06785

REVERSE OSMOSIS:

	total cap investment: 1.81092
	cat and chem cost: 0.01722
	electricity cost: 0.28871
	total fixed op cost: 0.02402

UV AOP:

	total cap investment: 3.66902
	cat and chem cost: 0.01295
	electricity cost: 0.01041
	total fixed op cost: 0.04886

CHLORINATION:

	total cap investment: 2.9143
	cat and chem cost: 0.00547
	electricity cost: 0.0
	total fixed op cost: 0.03881

MUNICIPAL DRINKING:

	total cap investment: 0.15869
	cat and chem cost: 0.0
	electricity cost: 0.03196
	total fixed op co

In [8]:
# RESET AREA TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [9]:
# Readjust recovery constraint and deactivate objective constraint
m.fs.objective_function.deactivate()

In [10]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


ONSITE STORAGE:

	total cap investment: 2.86121
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.03782

MICROFILTRATION:

	total cap investment: 5.09541
	cat and chem cost: 0.0
	electricity cost: 0.03405
	total fixed op cost: 0.06785

REVERSE OSMOSIS:

	total cap investment: 1.81092
	cat and chem cost: 0.01722
	electricity cost: 0.28871
	total fixed op cost: 0.02402

UV AOP:

	total cap investment: 3.66902
	cat and chem cost: 0.01295
	electricity cost: 0.01041
	total fixed op cost: 0.04886

CHLORINATION:

	total cap investment: 2.9143
	cat and chem cost: 0.00547
	electricity cost: 0.0
	total fixed op cost: 0.03881

MUNICIPAL DRINKING:

	total cap investment: 0.15869
	cat and chem cost: 0.0
	electricity cost: 0.03196
	total fixed op co

In [11]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])

In [12]:
###### SENSITIVITY ANALYSES 

In [13]:
# Find bounds for RO area:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).membrane_area[0])
        getattr(m.fs, key).membrane_area.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0] * 1, sense=env.minimize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).membrane_area[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).membrane_area[0]())

        getattr(m.fs, key).membrane_area.fix(stash_value)


----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 115.24967448159143
reverse_osmosis Minimum --> 10.0
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.1079239799011995
reverse_osmosis Maximum --> 2801.4480781762013


In [14]:
# Find bounds for RO pessure:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).feed.pressure[0])
        getattr(m.fs, key).feed.pressure.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.minimize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).feed.pressure[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).feed.pressure[0]())

        getattr(m.fs, key).feed.pressure.fix(stash_value)

----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 18.20986255616901
reverse_osmosis Minimum --> 5.039508494707099
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.1347178669870646
reverse_osmosis Maximum --> 93.87530402290179


In [15]:
m.fs.objective_function1.deactivate()

In [16]:
# run to rest before sensitivity
wt.run_water_tap(m = m, objective=False, skip_small = True)
print("LCOW -->", m.fs.costing.LCOW())

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.107587544445394


In [17]:
# sensitivity analyses
sens_df = pd.DataFrame()

lcow_list = []
water_recovery_list = []
scenario_value = []
scenario_name = []
elec_lcow = []

lcow_list.append(value(m.fs.costing.LCOW))
water_recovery_list.append(value(m.fs.costing.system_recovery))
scenario_value.append("n/a")
scenario_name.append("baseline")
elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))

runs_per_scenario = 20

############ onstream_factor 70-100% ############
stash_value = m.fs.costing_param.plant_cap_utilization
scenario = "plant_cap_utilization"
print("-------", scenario, "-------")
ub = 1
lb = 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.costing_param.plant_cap_utilization = i
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_cap_utilization = stash_value    
############################################################    

############ salinity  30-45 ############
stash_value = m.fs.seawater.conc_mass_in[0, "tds"]()
scenario = "inlet salinty 25k-45k mg/L"
print("-------", scenario, "-------")
ub = 25
lb = 45
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.seawater.conc_mass_in[0, "tds"].fix(i) 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.conc_mass_in[0, "tds"].fix(stash_value) 
############################################################

############ inlet flow ############
stash_value = m.fs.seawater.flow_vol_in[0]()
scenario = "inlet flow"
print("-------", scenario, "-------")
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.seawater.flow_vol_in.fix(i) 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.seawater.flow_vol_in.fix(stash_value) 
############################################################

############ lifetime years ############

stash_value = value(m.fs.costing_param.plant_lifetime_yrs)
scenario = "lifetime (yrs)"
print("-------", scenario, "-------")
ub = 50
lb = 15
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.costing_param.plant_lifetime_yrs = i 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.plant_lifetime_yrs = stash_value
############################################################

############ elec cost +-20% ############

stash_value = value(m.fs.costing_param.electricity_price)
scenario = "electricity price +- 20%"
print("-------", scenario, "-------")
ub = stash_value * 1.3
lb = stash_value * 0.7
step = (ub - lb) / runs_per_scenario
for i in np.arange(lb, ub + step, step):
    m.fs.costing_param.electricity_price = i 
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(i)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
m.fs.costing_param.electricity_price = stash_value
############################################################

############ RO scenarios --> pressure, membrane area, replacement rate% ############

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        area = value(getattr(m.fs, key).membrane_area[0])

        scenario_dict = {"membrane_area" : [-area*0.2, area*0.2], 
                         "pressure": [-15, 10], 
                         "factor_membrane_replacement": [-0.1, 0.1]}
        
        for scenario in scenario_dict.keys():
            if scenario == "pressure":
                stash_value = value(getattr(getattr(getattr(m.fs, key), "feed"), scenario)[0])
            else:
                stash_value = value(getattr(getattr(m.fs, key), scenario)[0])

            print("-------", scenario, "-------")
            ub = stash_value + scenario_dict[scenario][1]
            lb = stash_value + scenario_dict[scenario][0]
            step = (ub - lb) / runs_per_scenario
            
            for i in np.arange(lb, ub + step, step):
                if scenario == "pressure":
                    getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(i)
                else:
                    getattr(getattr(m.fs, key), scenario).fix(i)
                
                wt.run_water_tap(m = m, objective=False, skip_small = True)
                print("LCOW -->", m.fs.costing.LCOW())
    
                lcow_list.append(value(m.fs.costing.LCOW))
                water_recovery_list.append(value(m.fs.costing.system_recovery))
                scenario_value.append(i)
                scenario_name.append(key + "_" + scenario)
                elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    
            if scenario == "pressure":
                getattr(getattr(getattr(m.fs, key), "feed"), scenario).fix(stash_value)
            else:
                getattr(getattr(m.fs, key), scenario).fix(stash_value)
                            
############################################################

# final run to get baseline numbers again
wt.run_water_tap(m = m, objective=False, skip_small = True)

sens_df["lcow"] = lcow_list
sens_df["water_recovery"] =  water_recovery_list
sens_df["elec_lcow"] =  elec_lcow
sens_df["scenario_value"] = scenario_value
sens_df["scenario_name"] = scenario_name
sens_df["lcow_difference"] =  sens_df.lcow - value(m.fs.costing.LCOW)
sens_df["water_recovery_difference"] = (sens_df.water_recovery - value(m.fs.costing.system_recovery))
sens_df["elec_lcow_difference"] = (sens_df.elec_lcow - value(m.fs.costing.elec_frac_LCOW))

sens_df.elec_lcow = sens_df.elec_lcow * 100
sens_df.water_recovery = sens_df.water_recovery * 100


------- plant_cap_utilization -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.5822679206362773
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.549073488734817
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.517243211569033
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.4866946905307301
----------------------------------------------------------------------



AttributeError: '_ScalarFlowsheetBlock' object has no attribute 'seawater'

In [ ]:
sens_df.to_csv("results/case_studies/santa_barbara_baseline_sensitivity.csv")